In [75]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from data_encoders import *
from parametrizations import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
backend = Aer.get_backend('qasm_simulator')
n_samples = 4
n_features = 3
n_targets = 1

x = np.linspace(0, 2, n_samples)
y = x**2
y = y - np.min(y)
y = y/np.max(y)
y = np.pi/2 *y
y = y.reshape(-1,1) + 0.5

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2
X[:,2] = x**3
#X[:,3] = x**4



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

print(X)
print(y)

[[0.000 0.000 0.000]
 [1.047 0.349 0.116]
 [2.094 1.396 0.931]
 [3.142 3.142 3.142]]
[[0.500]
 [0.675]
 [1.198]
 [2.071]]


In [91]:
reps = 10
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

optimizer = Adam()
optimizer.initialize(model.theta.shape)

In [ ]:
for i in tqdm(range(100)):
    grad = [model.gradient(X, y)]
    grad = optimizer(grad)
    model.theta += 0.1*grad[0]
    y_pred = model.predict(X)
    mse = np.mean((y_pred - y)**2)
    loss = model.loss(X,y)
    #print(loss, model.theta, y_pred.flatten())
    print(f"{loss=:.3f}, {mse=:.3f}, {y_pred.flatten()=}")

  0%|          | 0/100 [00:00<?, ?it/s]

loss=0.351, mse=0.370, y_pred.flatten()=array([0.637, 1.090, 2.156, 1.460])
loss=0.403, mse=0.130, y_pred.flatten()=array([1.054, 0.722, 1.467, 1.698])
loss=0.404, mse=0.195, y_pred.flatten()=array([1.367, 0.554, 1.081, 2.086])
loss=0.395, mse=0.275, y_pred.flatten()=array([1.489, 0.619, 1.116, 2.405])
loss=0.390, mse=0.277, y_pred.flatten()=array([1.449, 0.642, 1.222, 2.524])
loss=0.391, mse=0.211, y_pred.flatten()=array([1.302, 0.529, 1.251, 2.494])
loss=0.405, mse=0.145, y_pred.flatten()=array([1.100, 0.343, 1.225, 2.403])
loss=0.415, mse=0.118, y_pred.flatten()=array([0.951, 0.213, 1.176, 2.308])
loss=0.416, mse=0.076, y_pred.flatten()=array([0.872, 0.306, 1.157, 2.240])
loss=0.414, mse=0.045, y_pred.flatten()=array([0.841, 0.478, 1.174, 2.223])
loss=0.407, mse=0.034, y_pred.flatten()=array([0.827, 0.615, 1.225, 2.228])
loss=0.407, mse=0.035, y_pred.flatten()=array([0.809, 0.689, 1.287, 2.263])
loss=0.411, mse=0.043, y_pred.flatten()=array([0.811, 0.703, 1.339, 2.308])
loss=0.418, 

# Cheese

In [ ]:
reps = 2
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

backend = Aer.get_backend('qasm_simulator')
n_samples = 2
n_features = 2
n_targets = 1

x = np.linspace(0, 2, n_samples)

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

y = model.predict(X)
print(y)

loss = model.loss(X,y)
print(loss)

In [ ]:
data = np.array([[0,0], [1,1], [2,2], [3,3]])
encoder = ParallelEncoder()


features = qk.QuantumRegister(2, name="features")
ancilla = qk.QuantumRegister(2, name="ancilla_f")
registers = [features, ancilla]
circuit = qk.QuantumCircuit(*registers)

circuit = encoder(circuit, features, ancilla, data)

circuit.draw()